In [1]:
import numpy as np
import pandas as pd
import random
from os import listdir

In [2]:
# 取出5种属性的所有属性值
def find_total_attrs(path):
	attrs = listdir(path)
	total_attrs = []
	for attr in attrs:
		vals = listdir(path + attr)
		vals= [ x.split('=')[1].split('.')[0] for x in vals ]
		if 'unknown' in vals:
			vals.remove('unknown')
		total_attrs.append(vals)
	# print(total_attrs)
	return total_attrs


# 一共5层，每层1~5种root cause,共25种
def find_root_causes(total_attrs):
	layers, kinds = 5, 5
	root_causes = []
	for i in range(layers):
		layer_vals = []
		for j in range(kinds):
			vals = []
			# 从5个属性里随机选择 i 个(第一层1个，第二层2个...)
			random_nums = random.sample([x for x in range(5)], i + 1)	
			for k in range(i + 1):
				vals.append( random.sample(total_attrs[random_nums[k]], 1)[0] )	
			layer_vals.append(vals)
			temp = layer_vals[:]		# 深拷贝
			root_causes.append(temp)
	return root_causes


## 随机找出25种根因

In [3]:
total_attrs = find_total_attrs('../data/result/')
root_causes = find_root_causes(total_attrs)

for root_cause in root_causes:
	print(root_cause)

FileNotFoundError: [Errno 2] No such file or directory: '../data/result/'

In [ ]:
# 计算异常
def compute_anomaly(path):
	o = open(path, 'rb')
	df = pd.read_csv(o)
	length = len(df)

	# 随机选取一个时间戳(为方便计算不选头尾处时间戳)
	rand_idx = random.randint(150, length - 150)
	timestamp = df.loc[rand_idx][0]						# 随机选择的timestamp
	temp_df = df[rand_idx - 144 : rand_idx + 144]		# 时间间隔为5分钟，共14天数据，选取一天的数据
	kpi = temp_df['KPI'].as_matrix()
	kpi = [float(i) for i in kpi]
	# print(kpi)
	mean = np.mean(kpi)
	std = np.std(kpi)
	total_anomaly = int(mean + 5*std)					# 此timestamp上的总KPI异常
	# print(total_anomaly)
	return timestamp, total_anomaly
	

# 分发异常
def distribute_anomaly(root_cause, path):
	timestamp, total_anomaly = compute_anomaly(path)
	print(root_cause, timestamp)
	path2 = '../data/origin/' + str(timestamp) + '.csv'
	path3 = '../data/anomaly/' + str(timestamp) + '.csv'
	for y in root_cause:
		for x in y:
			o = open(path2, 'rb')
			df = pd.read_csv(o, names  = ['i', 'e', 'c', 'p', 'l', 'kpi'])
			locs = []
			for idx in df.index:
				i, e, c, p, l = df.loc[idx]['i'], df.loc[idx]['e'], df.loc[idx]['c'], df.loc[idx]['p'], df.loc[idx]['l']
				if i in x or e in x or c in x or p in x or l in x:
					locs.append(idx)

			# 若此时间戳不含此属性值，则重新找一个时间戳
			if len(locs) == 0:
				distribute_anomaly(root_cause, path)
				return

			print(locs)
			anomaly = int(total_anomaly / len(locs))
			df_out = df[:]
			for idx in locs:
				i, e, c, p, l = df.loc[idx]['i'], df.loc[idx]['e'], df.loc[idx]['c'], df.loc[idx]['p'], df.loc[idx]['l']
				print(i, e, c, p, l)
				df_out.loc[idx] = {'i':i, 'e':e, 'c':c, 'p':p, 'l':l, 'kpi':df.loc[idx]['kpi'] + anomaly}
			csvfile = open(path3, 'w')
			df_out.to_csv(path3)

## 随机找出25个timestamp注入异常

In [ ]:
# 使用25种根因随机选择timestamp注入异常
path = '../data/KPI_new.csv'
distribute_anomaly(root_causes[0], path)

In [ ]:
path = '../data/anomaly/1536278100000.csv'
o = open(path, 'rb')
df = pd.read_csv(o)
df.head(20)

## 在第一个索引处插入异常老是插入列名，之后的索引处异常注入正确.